In [1]:
import torch
from transformers import AutoModelForQuestionAnswering, AutoTokenizer

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or 

In [2]:
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased-distilled-squad")

model = AutoModelForQuestionAnswering.from_pretrained("distilbert-base-uncased-distilled-squad")

In [7]:
def answer_questions(questions, text):
    for question in questions:
        inputs = tokenizer.encode_plus(question, text, return_tensors="pt")
        # Need to pop token type ids when using distilbert because this model does not 
        # handle them, but the encoder still sets them for some reason. 
        inputs.pop('token_type_ids', None)
        input_ids = inputs["input_ids"].tolist()[0]

        text_tokens = tokenizer.convert_ids_to_tokens(input_ids)
        answer_start_scores, answer_end_scores = model(**inputs)
        
        # Get the most likely beginning of answer with the argmax of the score
        answer_start = torch.argmax(answer_start_scores)
        # Get the most likely end of answer with the argmax of the score
        answer_end = torch.argmax(answer_end_scores) + 1  

        answer = tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(input_ids[answer_start:answer_end]))

        print(f"Question: {question}")
        print(f"Answer: {answer}\n")

In [8]:
text = '''My favorite color is red. My favorite color is NOT blue. There are 100 million neurons in the brain. There are infinitely many prime numbers. The Cure are an English rock band formed in Crawley, West Sussex, in 1978.'''
questions = ['When did The Cure form?', 'How many prime numbers are there?', 
             'What is my favorite color', 'When did Lincoln die?']
answer_questions(questions, text)

Question: When did The Cure form?
Answer: an english rock band formed in crawley , west sussex , in 1978 .

Question: How many prime numbers are there?
Answer: infinitely many

Question: What is my favorite color
Answer: not blue

Question: When did Lincoln die?
Answer: my favorite color is not blue .

